In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
import utils
from pandas import Timedelta
import time
from utils import logger, sql_to_df, run_sql, session, conn
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

09:55:06 INFO:Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.16, Platform: macOS-10.15.7-x86_64-i386-64bit
09:55:06 INFO:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:55:06 INFO:Setting use_openssl_only mode to False
09:55:07 INFO:Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.16, Platform: macOS-10.15.7-x86_64-i386-64bit
09:55:07 INFO:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
09:55:08 INFO:Snowpark Session information: 
"version" : 1.3.0,
"python.version" : 3.8.16,
"python.connector.version" : 2.7.12,
"python.connector.session.id" : 876302180442122,
"os.name

connected to account omwykhw-jx58967 wh OPS db SNOWFLAKE_USAGE schema SANDBOX with role DEV
time: 18.2 ms (started: 2023-04-27 09:55:08 -04:00)


In [3]:
import snowflake.connector
import threading
import time
import os
class WarehousePoller:
    def __init__(self):
        # self.account = account
        # self.user = user
        # self.password = password
        self.connection = None
        self.polling_thread = None
        print("Number of active threads:", threading.active_count())

    def connect(self):
        self.connection = snowflake.connector.connect(
            user=os.environ['SNOWFLAKE_USER'],
            role=os.environ['SNOWFLAKE_ROLE'],
            password=os.environ['SNOWFLAKE_PASSWORD'],
            account=os.environ['SNOWFLAKE_ACCOUNT'],
            warehouse=os.environ['SNOWFLAKE_WAREHOUSE'],
            database=os.environ['SNOWFLAKE_DATABASE'],
            schema=os.environ['SNOWFLAKE_SCHEMA'],
            client_session_keep_alive=True
        )


    def disconnect(self):
        if self.connection is not None:
            self.connection.close()
            self.connection = None

    def poll_warehouses(self):
        while True:
            if self.connection is not None:
                cursor = self.connection.cursor()
                cursor.execute("show warehouses")
                rows = cursor.fetchall()
                print(f"Available warehouses: {rows}")
                cursor.close()
            time.sleep(10)

    def start_polling(self):
        self.polling_thread = threading.Thread(target=self.poll_warehouses)
        self.polling_thread.start()

    def stop_polling(self):
        if self.polling_thread is not None:
            self.polling_thread.join()
            self.polling_thread = None

time: 1.73 ms (started: 2023-04-27 09:55:08 -04:00)


In [4]:
poller = WarehousePoller()
poller.connect()

09:55:08 INFO:Snowflake Connector for Python Version: 2.7.12, Python Version: 3.8.16, Platform: macOS-10.15.7-x86_64-i386-64bit
09:55:08 INFO:This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.


Number of active threads: 9
time: 516 ms (started: 2023-04-27 09:55:08 -04:00)


In [5]:
poller.start_polling()


09:55:17 INFO:query: [show warehouses]


time: 1.53 ms (started: 2023-04-27 09:55:17 -04:00)


09:55:17 INFO:query execution done


Available warehouses: [('AUTOSUSPEND_SIXTY', 'SUSPENDED', 'STANDARD', 'X-Small', 0, 0, 'N', 'N', 60, 'true', '', '', '', '', datetime.datetime(2023, 4, 11, 8, 13, 27, 902000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 11, 8, 13, 28, 746000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 11, 8, 13, 28, 746000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWFLAKE_USAGE', '', 'null', 0, 0, 0, 1, '52231856'), ('DEMO', 'SUSPENDED', 'STANDARD', 'X-Small', 0, 0, 'N', 'N', 37, 'true', '', '', '', '', datetime.datetime(2022, 11, 9, 7, 31, 26, 305000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), datetime.datetime(2023, 4, 26, 17, 12, 39, 999000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 26, 17, 12, 39, 999000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWFLAKE_USAGE', '', 'nu

In [6]:
print("Number of active threads:", threading.active_count())

Number of active threads: 11
time: 1.14 ms (started: 2023-04-27 09:55:20 -04:00)


In [ ]:

# Wait a bit to see the output
# time.sleep(60)

poller.stop_polling()
poller.disconnect()

09:55:27 INFO:query: [show warehouses]
09:55:27 INFO:query execution done


Available warehouses: [('AUTOSUSPEND_SIXTY', 'SUSPENDED', 'STANDARD', 'X-Small', 0, 0, 'N', 'N', 60, 'true', '', '', '', '', datetime.datetime(2023, 4, 11, 8, 13, 27, 902000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 11, 8, 13, 28, 746000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 11, 8, 13, 28, 746000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWFLAKE_USAGE', '', 'null', 0, 0, 0, 1, '52231856'), ('DEMO', 'SUSPENDED', 'STANDARD', 'X-Small', 0, 0, 'N', 'N', 37, 'true', '', '', '', '', datetime.datetime(2022, 11, 9, 7, 31, 26, 305000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), datetime.datetime(2023, 4, 26, 17, 12, 39, 999000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 26, 17, 12, 39, 999000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWFLAKE_USAGE', '', 'nu

09:55:37 INFO:query: [show warehouses]
09:55:37 INFO:query execution done


Available warehouses: [('AUTOSUSPEND_SIXTY', 'SUSPENDED', 'STANDARD', 'X-Small', 0, 0, 'N', 'N', 60, 'true', '', '', '', '', datetime.datetime(2023, 4, 11, 8, 13, 27, 902000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 11, 8, 13, 28, 746000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 11, 8, 13, 28, 746000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWFLAKE_USAGE', '', 'null', 0, 0, 0, 1, '52231856'), ('DEMO', 'SUSPENDED', 'STANDARD', 'X-Small', 0, 0, 'N', 'N', 37, 'true', '', '', '', '', datetime.datetime(2022, 11, 9, 7, 31, 26, 305000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), datetime.datetime(2023, 4, 26, 17, 12, 39, 999000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 26, 17, 12, 39, 999000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWFLAKE_USAGE', '', 'nu

09:55:47 INFO:query: [show warehouses]
09:55:48 INFO:query execution done


Available warehouses: [('AUTOSUSPEND_SIXTY', 'SUSPENDED', 'STANDARD', 'X-Small', 0, 0, 'N', 'N', 60, 'true', '', '', '', '', datetime.datetime(2023, 4, 11, 8, 13, 27, 902000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 11, 8, 13, 28, 746000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 11, 8, 13, 28, 746000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWFLAKE_USAGE', '', 'null', 0, 0, 0, 1, '52231856'), ('DEMO', 'SUSPENDED', 'STANDARD', 'X-Small', 0, 0, 'N', 'N', 37, 'true', '', '', '', '', datetime.datetime(2022, 11, 9, 7, 31, 26, 305000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), datetime.datetime(2023, 4, 26, 17, 12, 39, 999000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), datetime.datetime(2023, 4, 26, 17, 12, 39, 999000, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>), 'SNOWFLAKE_USAGE', '', 'nu